# User features

## Import packages

In [40]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
random.seed(32)
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Import data

In [70]:
authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')

pre_df = pd.read_pickle("preprocessed.pkl")
df = pre_df.set_index(['index', 'author'])
df.head(10)
# df.info(verbose=True)

,,body,probody,tokens,senttokens,agreeableness,openness,conscientiousness,extraversion,neuroticism,agree,...,link_id,parent_id,score,controversiality,gilded,id,subreddit,ups,word_count,word_count_quoteless
index,author,,,,,,,,,,,,,,,,,,,,,
1,Sabata11792,That's subtle enough to just look like a coinc...,subtle enough look like,"[subtl, enough, look, like]",[That's subtle enough to just look like a coin...,8.0,11.0,74.0,1.0,25.0,0,...,t3_3sdrrj,t1_cwwel0w,4.0,0.0,0.0,cwwn8s9,mylittlepony,4.0,9.0,9.0
21,Shadow_Of_,"Downturned nose, dirty skin, tattoos, small ch...",downturned dirty small small obvious would not,"[downturn, dirti, small, small, obviou, would,...","[Downturned nose, dirty skin, tattoos, small c...",76.0,47.0,1.0,4.0,75.0,1,...,t3_68roag,t3_68roag,1.0,0.0,0.0,dh13z6f,CringeAnarchy,0.0,15.0,15.0
34,xenomouse,"Yes, if I was a man they'd call it a man cave....",man would call man guess mouse agree not safe ...,"[man, would, call, man, guess, mous, agre, not...","[Yes, if I was a man they'd call it a man cave...",26.0,93.0,49.0,70.0,16.0,0,...,t3_73lfuz,t3_73lfuz,2.0,0.0,0.0,dnrdhmc,infj,0.0,45.0,45.0
37,eiznekk,Added you back! Thank you :D,added thank,"[ad, thank]","[Added you back!, Thank you :D]",70.0,64.0,5.0,5.0,95.0,1,...,t3_3i8rel,t1_cue9ixy,1.0,0.0,0.0,cue9pjb,friendsafari,1.0,6.0,6.0
62,vitrael2,I squatted 225x14 a couple weeks ago and I mad...,squatted 225x14 couple weeks ago made sad card...,"[squat, 225x14, coupl, week, ago, made, sad, c...",[I squatted 225x14 a couple weeks ago and I ma...,26.0,98.0,75.0,93.0,29.0,0,...,t3_2wwmlc,t3_2wwmlc,8.0,0.0,0.0,courxwo,Weakpots,8.0,20.0,20.0
66,Lastrevio,"I don't like this description by the way, too ...",not like description vague disorganized short ...,"[not, like, descript, vagu, disorgan, short, a...","[I don't like this description by the way, too...",3.0,74.0,4.0,46.0,92.0,0,...,t3_6talqt,t3_6talqt,6.0,0.0,0.0,dlj7nbh,JungianTypology,0.0,28.0,28.0
70,Lastrevio,nothing,nothing,[noth],[nothing],3.0,74.0,4.0,46.0,92.0,0,...,t3_6615dh,t3_6615dh,2.0,0.0,0.0,dgf7lzx,mbti,0.0,1.0,1.0
71,Lastrevio,filthy sensor,filthy sensor,"[filthi, sensor]",[filthy sensor],3.0,74.0,4.0,46.0,92.0,0,...,t3_718iq8,t1_dn8zv65,7.0,0.0,0.0,dn9rye2,mbti,0.0,2.0,2.0
75,Lastrevio,This is how COGFUNC_MENTION dom brain works lol,dom brain works lol,"[dom, brain, work, lol]",[This is how COGFUNC_MENTION dom brain works lol],3.0,74.0,4.0,46.0,92.0,0,...,t3_6p5t7m,t3_6p5t7m,1.0,0.0,0.0,dkpfm6x,entp,0.0,8.0,8.0


In [25]:
# df['subreddit']

0              mylittlepony
1             CringeAnarchy
2                      infj
3              friendsafari
4                  Weakpots
              ...          
82                Overwatch
83                teenagers
84                     intj
85                     mbti
86    Random_Acts_Of_Amazon
Name: subreddit, Length: 87, dtype: object

In [26]:
authors.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10295 entries, 0 to 10294
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   author                     10295 non-null  object 
 1   mbti                       9084 non-null   object 
 2   introverted                9078 non-null   float64
 3   intuitive                  9083 non-null   float64
 4   thinking                   9080 non-null   float64
 5   perceiving                 9074 non-null   float64
 6   gender                     3227 non-null   object 
 7   age                        2324 non-null   float64
 8   enneagram                  794 non-null    object 
 9   country                    2146 non-null   object 
 10  state                      857 non-null    object 
 11  type                       1611 non-null   object 
 12  agreeableness              1606 non-null   float64
 13  openness                   1588 non-null   flo

In [27]:
# df.groupby('author').sum()

,index,agreeableness,openness,conscientiousness,extraversion,neuroticism,agree,openn,consc,extra,neuro,language,downs,created_utc,score,controversiality,gilded,ups,word_count,word_count_quoteless
author,,,,,,,,,,,,,,,,,,,,
AlmightyWibble,137,69.0,11.0,1.0,34.0,99.0,1,0,0,0,1,0,0.0,1.431453e+09,1.0,0.0,0.0,1.0,38.0,38.0
Ambedo_1,962,11.0,6.0,61.0,1.0,45.0,0,0,1,0,0,0,0.0,1.479464e+09,1.0,0.0,0.0,0.0,120.0,120.0
BadgerKid96,944,77.0,73.0,73.0,1.0,98.0,1,1,1,0,1,0,0.0,1.469684e+09,1.0,0.0,0.0,1.0,1.0,1.0
Bliss86,584,70.0,34.0,18.0,58.0,9.0,1,0,0,1,0,0,0.0,1.424874e+09,1.0,0.0,0.0,1.0,21.0,21.0
Blitz179,865,93.0,34.0,9.0,15.0,75.0,1,0,0,0,1,0,0.0,1.425445e+09,4.0,0.0,0.0,4.0,9.0,9.0
BubblesAndSass,494,91.0,84.0,80.0,74.0,25.0,1,1,1,1,0,1,0.0,1.478046e+09,2.0,0.0,0.0,0.0,2.0,2.0
BurnedOutInAJar,721,70.0,96.0,15.0,37.0,80.0,1,1,0,0,1,0,0.0,1.454092e+09,1.0,0.0,0.0,1.0,28.0,28.0
Cocktorpedo,1611,62.0,190.0,2.0,68.0,198.0,0,2,0,0,2,2,0.0,2.859601e+09,15.0,0.0,0.0,15.0,23.0,22.0
CrateredMoon,911,10.0,78.0,16.0,30.0,36.0,0,1,0,0,0,0,0.0,1.423175e+09,1.0,0.0,0.0,1.0,21.0,21.0


## LDA

In [58]:
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

def apply_lda(df, number, name):
    lst = []
    # filter out stopwords including negations for topic modeling
    stopwordList = stopwords.words('english')
    df['ldatokens'] = df['probody'].apply(lambda x: ' '.join([x for x in x.split() if (x not in stopwordList)]))
    df['ldatokens'] = df['ldatokens'].apply(lambda x: (word_tokenize(x)))
    for row in df['ldatokens']:
        if len(row) == 0:
            lst.append([(0, '0*"X"'), (1, '0*"X"'), (2, '0*"X"')])
        else:
            texts = [list(token) for token in row]
            dictionary = corpora.Dictionary(texts)
            corpus = [dictionary.doc2bow(text) for text in texts]
            ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=number, id2word = dictionary, passes=20)
            result = ldamodel.print_topics(num_topics=number, num_words=1)
            lst.append(result)
    df[name] = lst
    return df

test = apply_lda(df, 3, "ldathree")
test['ldathree']

0     [(0, 0.292*"o"), (1, 0.167*"e"), (2, 0.134*"u")]
1     [(0, 0.261*"l"), (1, 0.194*"o"), (2, 0.168*"d")]
2     [(0, 0.240*"e"), (1, 0.168*"a"), (2, 0.216*"u")]
3     [(0, 0.455*"d"), (1, 0.182*"a"), (2, 0.144*"a")]
4     [(0, 0.169*"o"), (1, 0.083*"2"), (2, 0.174*"e")]
                            ...                       
82    [(0, 0.180*"l"), (1, 0.043*"v"), (2, 0.186*"e")]
83                [(0, 0*"X"), (1, 0*"X"), (2, 0*"X")]
84    [(0, 0.174*"n"), (1, 0.213*"a"), (2, 0.301*"e")]
85    [(0, 0.120*"t"), (1, 0.116*"s"), (2, 0.141*"c")]
86    [(0, 0.220*"o"), (1, 0.206*"e"), (2, 0.186*"i")]
Name: ldathree, Length: 87, dtype: object

# Wrapper

In [21]:
def extract_userfeatures(df):
    print(1)